In [1]:
from langchain_core.runnables import Runnable, RunnableLambda
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.tools import tool
from langchain_core.output_parsers import PydanticOutputParser
from langchain.output_parsers.enum import EnumOutputParser
from langchain_openai import ChatOpenAI
from langfuse.decorators import observe
from pydantic import BaseModel, Field
from enum import Enum

In [2]:
my_string_prompt = PromptTemplate.from_template("""
Tell me a joke about {topic}
""")

my_chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "you are a helpful assistant"),
    ("human", "tell me a joke about {topic}")
])

In [3]:
model = ChatOpenAI()

In [4]:
chain = my_string_prompt | model

In [5]:
chain.invoke(input={
    "topic": "cats"
})

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}